In [ ]:
#!pip install prefect-ray

In [1]:
import ray
from ray import tune
import pandas as pd
import warnings
import math
import time
import os
from typing import List

# Suppress noisy requests warnings.
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [2]:
ray.init()

2022-12-14 19:48:29,292	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67100672 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=8.10gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2022-12-14 19:48:29,434	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.10.4
Ray version:,2.2.0


In [ ]:
import boto3
s3 = boto3.resource('s3')
# Download the file from S3.
bucket = 'nyc-tlc'
s3.Bucket(bucket).download_file('csv_backup/yellow_tripdata_2011-04.csv', '../data/yellow_tripdata_2011-04.csv')
s3.Bucket(bucket).download_file('csv_backup/yellow_tripdata_2009-12.csv', '../data/yellow_tripdata_2009-12.csv')
s3.Bucket(bucket).download_file('csv_backup/yellow_tripdata_2010-06.csv', '../data/yellow_tripdata_2010-06.csv')
s3.Bucket(bucket).download_file('csv_backup/yellow_tripdata_2010-07.csv', '../data/yellow_tripdata_2010-07.csv')

In [ ]:
direct = '../data/'
list_files = os.listdir(direct)
list_filtered = []
for file in list_files:
    if file.startswith('yellow_tripdata'):
        ds_csv = ray.data.read_csv(os.path.abspath('../data/' + file))
        ds_csv.repartition(12).write_parquet(f'../data/parquet/{file[:-4]}')
        del ds_csv

In [4]:
ds = ray.data.read_parquet('../data/parquet/')

Parquet Files Sample: 100%|██████████| 2/2 [00:00<00:00,  8.08it/s]


In [5]:
# Create 10 blocks for parallelism
ds = ds.repartition(10)

Repartition:   0%|          | 0/10 [00:00<?, ?it/s](raylet) Spilled 2266 MiB, 35 objects, write throughput 360 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 7040 MiB, 113 objects, write throughput 602 MiB/s.
Repartition: 100%|██████████| 10/10 [00:23<00:00,  2.35s/it]


In [6]:
total_rows = ds.count()
print('Total rows: {:,}'.format(total_rows))

Total rows: 44,927,917


In [7]:
ds.limit(5).to_pandas()

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,VTS,2009-12-17 07:35:00,2009-12-17 07:40:00,1,0.11,-73.987928,40.737885,None,NaN,-73.990335,40.748450,Credit,4.9,0.0,0.5,1.0,0.0,6.4
1,VTS,2009-12-21 14:19:00,2009-12-21 14:24:00,1,1.07,-73.956008,40.779558,None,NaN,-73.967303,40.787833,CASH,4.9,0.0,0.5,0.0,0.0,5.4
2,VTS,2009-12-18 03:09:00,2009-12-18 03:34:00,1,8.98,-73.955745,40.689503,None,NaN,-73.937730,40.737463,CASH,23.7,0.5,0.5,0.0,0.0,24.7
3,VTS,2009-12-14 21:24:00,2009-12-14 21:33:00,2,1.66,-73.983985,40.754645,None,NaN,-73.986195,40.737610,Credit,6.9,0.5,0.5,3.0,0.0,10.9
4,VTS,2009-12-18 08:17:00,2009-12-18 08:29:00,1,1.55,-73.959132,40.769265,None,NaN,-73.976267,40.760615,CASH,7.7,0.0,0.5,0.0,0.0,8.2


In [44]:
def transform_batch(df: pd.DataFrame) -> pd.DataFrame:
    return df[(df["Trip_Distance"] > 2) & (df["Passenger_Count"] > 2)]

transformed_ds = ds.map_batches(transform_batch)

Map_Batches: 100%|██████████| 10/10 [00:07<00:00,  1.31it/s]


In [45]:
transformed_ds.limit(10).to_pandas()

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,VTS,2009-12-21 12:16:00,2009-12-21 12:52:00,3,16.57,-73.790967,40.645922,None,NaN,-73.971593,40.752568,Credit,45.0,0.0,0.5,7.0,4.57,57.07
1,VTS,2009-12-16 01:31:00,2009-12-16 01:47:00,5,5.03,-74.003327,40.751415,None,NaN,-73.990005,40.690455,Credit,13.7,0.5,0.5,3.0,0.00,17.70
2,VTS,2009-12-20 19:57:00,2009-12-20 20:13:00,5,3.22,-73.994107,40.751255,None,NaN,-73.970408,40.789250,Credit,10.9,0.0,0.5,3.0,0.00,14.40
3,VTS,2009-12-14 22:55:00,2009-12-14 23:04:00,5,2.24,-73.991612,40.759822,None,NaN,-74.003780,40.737545,CASH,7.7,0.5,0.5,0.0,0.00,8.70
4,VTS,2009-12-21 08:26:00,2009-12-21 09:07:00,5,4.49,-73.974723,40.792132,None,NaN,-73.977962,40.748085,CASH,21.7,0.0,0.5,0.0,0.00,22.20
5,VTS,2009-12-21 05:40:00,2009-12-21 05:45:00,5,2.17,-73.971808,40.764552,None,NaN,-73.992482,40.740508,CASH,6.9,0.5,0.5,0.0,0.00,7.90
6,VTS,2009-12-17 14:17:00,2009-12-17 15:19:00,3,17.74,-73.986025,40.762557,None,NaN,-73.789580,40.642812,CASH,45.0,0.0,0.5,0.0,4.57,50.07
7,VTS,2009-12-14 20:21:00,2009-12-14 20:34:00,5,2.09,-73.965510,40.768725,None,NaN,-73.972115,40.746033,CASH,8.9,0.5,0.5,0.0,0.00,9.90
8,VTS,2009-12-21 07:32:00,2009-12-21 08:00:00,3,13.32,-74.015207,40.709402,None,NaN,-73.873348,40.774312,CASH,31.3,0.0,0.5,0.0,0.00,31.80
9,VTS,2009-12-26 18:04:00,2009-12-26 18:18:00,5,4.26,-73.995898,40.738567,None,NaN,-73.977603,40.787870,CASH,12.1,0.0,0.5,0.0,0.00,12.60


In [47]:
total_rows = transformed_ds.count()
print('Total rows: {:,}'.format(total_rows))

Total rows: 1,008,808


In [8]:
def convert_size(size_bytes):
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

total_bytes = ds.size_bytes()
print("Total size: {}".format(convert_size(total_bytes)))

Total size: 5.52 GB


In [12]:
# What's the longets trip distance, largest tip amount, and most number of passengers?
ds.max(["Trip_Distance", "Tip_Amt", "Passenger_Count"])

Shuffle Reduce: 100%|██████████| 1/1 [00:00<00:00, 118.20it/s]


ArrowRow({'max(Trip_Distance)': 50.0,
          'max(Tip_Amt)': 100.0,
          'max(Passenger_Count)': 208})

In [13]:
@ray.remote
class Worker:
    def __init__(self, rank: int):
        pass

    def train(self, shard) -> int:
        for batch in shard.iter_batches(batch_size=256):
            pass
        return shard.count()

In [14]:
workers = [Worker.remote(i) for i in range(4)]
shards = ds.split(n=4, locality_hints=workers)

In [16]:
ray.get([w.train.remote(s) for w, s in zip(workers, shards)])

[13478376, 13478375, 8985583, 8985583]

In [29]:
@ray.remote
class Worker:
    pass

# Explicitly stop
ray.shutdown()
assert not ray.is_initialized()

In [31]:
ray.init(num_cpus=32)

2022-12-14 19:23:48,925	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67096576 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=8.18gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2022-12-14 19:23:49,074	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.10.4
Ray version:,2.2.0


In [3]:
ds = ray.data.read_parquet('../data/parquet/yellow_tripdata_2011-04').repartition(10)

Repartition: 100%|██████████| 10/10 [00:00<00:00, 11.60it/s]


In [38]:
def objective(*args):
    ds.limit(10).show()

In [37]:
tuner = tune.Tuner(
    tune.with_resources(objective, {"cpu": 1}),
    tune_config=tune.TuneConfig(
        num_samples=1,
        max_concurrent_trials=32
    )
)
tuner.fit()

2022-12-14 19:26:20,726	WARNING callback.py:108 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial objective_3059b_00000 completed. Last result: 


2022-12-14 19:26:22,466	INFO tune.py:762 -- Total run time: 1.75 seconds (1.63 seconds for the tuning loop).


(objective pid=10495) {'vendor_id': 'VTS', 'pickup_datetime': datetime.datetime(2011, 4, 29, 5, 50), 'dropoff_datetime': datetime.datetime(2011, 4, 29, 5, 55), 'passenger_count': 1, 'trip_distance': 2.15, 'pickup_longitude': -73.940545, 'pickup_latitude': 40.83491, 'rate_code': 1, 'store_and_fwd_flag': '', 'dropoff_longitude': -73.942723, 'dropoff_latitude': 40.811692, 'payment_type': 'CSH', 'fare_amount': 6.9, 'surcharge': 0.5, 'mta_tax': 0.5, 'tip_amount': 0.0, 'tolls_amount': 0.0, 'total_amount': 7.9}
(objective pid=10495) {'vendor_id': 'VTS', 'pickup_datetime': datetime.datetime(2011, 4, 25, 9, 13), 'dropoff_datetime': datetime.datetime(2011, 4, 25, 9, 25), 'passenger_count': 1, 'trip_distance': 2.39, 'pickup_longitude': -73.957882, 'pickup_latitude': 40.773793, 'rate_code': 1, 'store_and_fwd_flag': '', 'dropoff_longitude': -73.981958, 'dropoff_latitude': 40.76717, 'payment_type': 'CRD', 'fare_amount': 8.9, 'surcharge': 0.0, 'mta_tax': 0.5, 'tip_amount': 1.0, 'tolls_amount': 0.0, '

In [4]:
ds = ds.lazy()